<a href="https://colab.research.google.com/github/muhhanifra/Shoppee-product-Classification-Challenge/blob/main/resnet_inception_v2_shopee_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# InceptionResNetV2

In [ ]:
from google.colab import drive
import zipfile
#open drive
drive.mount('/content/gdrive')
#unzip data
with zipfile.ZipFile('/content/gdrive/My Drive/Copy of shopee-product-detection-dataset.zip') as zip_file:
  zip_file.extractall('/content/dataset')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Input, Activation, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.utils import plot_model
import numpy as np
from google.colab import drive
import pandas as pd
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import shutil

In [ ]:
# freeze layer 0 to layer l, and unfreeze layer l+1 up until layer L
def set_trainable(model, transfer_criteria = 'finetune_all', print_details = False):

  #freeze all layers
  if transfer_criteria == 'freeze_all':
    for layer in model.layers:
      layer.trainable = False
      
      if print_details == True:
        print('layer name: {}, trainable {}'.format(layer.name, layer.trainable))

  #finetune all layers
  elif transfer_criteria == 'finetune_all':
    for layer in model.layers:
      layer.trainable = True

      if print_details == True:
        print('layer name: {}, trainable {}'.format(layer.name, layer.trainable))
  
  #set layers trainable from the specified layer up to the last layer
  else:
    trainable_condition = False
    for layer in model.layers:
      if layer.name == transfer_criteria:
        trainable_condition = True
      layer.trainable = trainable_condition

      if print_details == True:
        print('layer name: {}, trainable {}'.format(layer.name, layer.trainable))

In [ ]:
def plt_model(model):
  plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

def create_transfer_learning_model(transferred_model):
  
  i = tf.keras.layers.Input([224, 224, 3], dtype = tf.uint8)
  _i = tf.cast(i, tf.float32)
  _i = tf.keras.applications.inception_resnet_v2.preprocess_input(_i)
  
  core = transferred_model(_i) 
  x = layers.GlobalAveragePooling2D()(core)
  x = Dense(42, activation='softmax')(x)
  model = Model(inputs=[i], outputs=[x])

  return model

In [ ]:
model_name = 'resnet_inception_v2_cont_1.h5'

In [ ]:
resnet_inception = InceptionResNetV2(include_top=False,
                                     weights="imagenet",
                                     input_tensor=None,
                                     input_shape = (224, 224, 3),
                                     pooling=None)

set_trainable(resnet_inception, transfer_criteria = 'block8_8_mixed', print_details = True)
model = create_transfer_learning_model(resnet_inception)

In [ ]:
# model_path = '/content/gdrive/My Drive/resnet_inception_v2/resnet_inception_v2/resnet_inception_v2.h5'
# model = tf.keras.models.load_model(model_path)
# set_trainable(model.layers[4], transfer_criteria = 'block8_8_mixed', print_details = True)

In [ ]:
#-----hyperparams
EPOCHS = 7
BATCH_SIZE = 64
learning_rate = 0.00005

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.epoch = 0 
        self.per_batch_losses = []
        self.per_batch_acc = []

    def on_batch_end(self, batch, logs):
        
        self.per_batch_losses.append(logs.get("loss"))
        self.per_batch_acc.append(logs.get('accuracy'))
        
    def on_epoch_begin(self, batch, logs):
        self.epoch = self.epoch + 1
        # if self.epoch == 5:
        #   model.optimizer.lr = 0.0003
        #   print('proceed with smaller learning rate...\n')
        # if self.epoch == 8:
        #   model.optimizer.lr = 0.0001
        # if self.epoch == 10:
        #   print('proceed with even smaller learning rate...\n')
        #   model.optimizer.lr = 0.00005


step_hist = LossHistory()

es = EarlyStopping(monitor='val_loss', 
                   mode='min', 
                   verbose=1, 
                   patience=30)

mc = ModelCheckpoint(model_name, 
                     monitor='val_accuracy', 
                     mode='max', 
                     verbose=1,
                     save_best_only=True)

opt = keras.optimizers.Nadam(lr = learning_rate)
model.compile(optimizer= opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
train_dir = '/content/dataset/train/train'


datagen = ImageDataGenerator(width_shift_range= 0.2,
                             height_shift_range = 0.2,
                             horizontal_flip = True,
                             zoom_range=0.15,
                             shear_range = 15,
                             rotation_range = 20, 
                             validation_split = 0.1,
                             brightness_range=[0.2,1.0])

data_flow_train = datagen.flow_from_directory(train_dir,
                                        target_size= (224,224), 
                                        class_mode = 'categorical',
                                        batch_size = BATCH_SIZE, 
                                        subset = 'training',
                                        seed=12)

data_flow_val = datagen.flow_from_directory(train_dir,
                                        target_size= (224,224), 
                                        class_mode = 'categorical',
                                        batch_size = BATCH_SIZE, 
                                        subset = 'validation',
                                        seed=12)
m_train = data_flow_train.samples
m_val = data_flow_val.samples

Found 94869 images belonging to 42 classes.
Found 10523 images belonging to 42 classes.


In [ ]:
def save_model_to_drive(model_name):
  model_name_string = model_name.split('.')[0]
  destination = '/content/gdrive/My Drive/resnet_inception_v2/{}'.format(model_name_string)
  if not os.path.exists(destination):
      os.makedirs(destination)

  source_pretrained = '/content/{}'.format(model_name)

  shutil.move(source_pretrained, destination)

In [ ]:
try:
  history = model.fit(data_flow_train, 
                      validation_data= data_flow_val,
                      validation_steps = 10, 
                      steps_per_epoch = m_train/BATCH_SIZE, 
                      epochs = EPOCHS, 
                      callbacks=[es, mc, step_hist])
except:
  print('something went wrong during training!, saving the last model to drive')
  save_model_to_drive(model_name)

Epoch 1/7
1483/1482 [==============================] - ETA: 0s - loss: 0.7115 - accuracy: 0.7887
Epoch 00001: val_accuracy improved from -inf to 0.72344, saving model to resnet_inception_v2_cont_1.h5
1483/1482 [==============================] - 2081s 1s/step - loss: 0.7115 - accuracy: 0.7887 - val_loss: 1.0512 - val_accuracy: 0.7234
Epoch 2/7
1483/1482 [==============================] - ETA: 0s - loss: 0.6643 - accuracy: 0.8028
Epoch 00002: val_accuracy improved from 0.72344 to 0.72969, saving model to resnet_inception_v2_cont_1.h5
1483/1482 [==============================] - 2083s 1s/step - loss: 0.6643 - accuracy: 0.8028 - val_loss: 1.0253 - val_accuracy: 0.7297
Epoch 3/7
1483/1482 [==============================] - ETA: 0s - loss: 0.6384 - accuracy: 0.8103
Epoch 00003: val_accuracy did not improve from 0.72969
1483/1482 [==============================] - 2084s 1s/step - loss: 0.6384 - accuracy: 0.8103 - val_loss: 1.0707 - val_accuracy: 0.7234
Epoch 4/7
1483/1482 [===================

In [ ]:
try:
  save_model_to_drive(model_name)
except:
  print('ERROR ! could not find the saved model')

In [ ]:
def learning_history_recap(model_name, history, step_hist):
  df_history = pd.DataFrame(history.history)
  per_batch_hist = {'loss': step_hist.per_batch_losses,
                    'acc': step_hist.per_batch_acc}
  df_perbatch_hist = pd.DataFrame(per_batch_hist)
  
  df_history.to_csv('{}_per_epoch_recap.csv'.format(model_name),index = False)
  df_perbatch_hist.to_csv('{}_per_batch_recap.csv'.format(model_name),index = False)

  return (df_history, df_perbatch_hist)

In [ ]:
model_name_string = model_name.split('.')[0]
destination = '/content/gdrive/My Drive/resnet_inception_v2/{}'.format(model_name_string)

learning_recaps = learning_history_recap(model_name, history, step_hist)
history_source = '/content/{}_per_epoch_recap.csv'.format(model_name)
history_per_batch_source = '/content/{}_per_batch_recap.csv'.format(model_name)

shutil.move(history_source, destination)
shutil.move(history_per_batch_source, destination)

In [ ]:
plt.figure(figsize = (15,8))
plt.plot(step_hist.per_batch_losses)
plt.ylabel('loss')
plt.show()

plt.figure(figsize = (15,8))
plt.plot(step_hist.per_batch_acc)
plt.ylabel('accuracy')
plt.show()